<a href="https://colab.research.google.com/github/lvllvl/parallel_processing/blob/main/luminance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -qq install -y libsm6 libxext6 libopencv-dev
!apt-get install -y libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%%writefile luminance.cu

#include <stdio.h>
#include <opencv2/opencv.hpp>

__global__ void computeLuminance( unsigned char* input, float* output, int width, int height ){

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if ( x < width && y < height ) {
        int index = y * width + x;
        unsigned char r = input[ 3 * index ];
        unsigned char g = input[ 3 * index + 1 ];
        unsigned char b = input[ 3 * index + 2 ];
        output[ index ] = 0.299f * r + 0.587f * g + 0.114f * b;
    }
}

int main() {
    cv::Mat img = cv::imread( "/content/drive/MyDrive/images/bright_color_photo.png", cv::IMREAD_COLOR );
    if ( img.empty() ) {
        printf( "Failed to open image\n" );
        return -1;
    }

    unsigned char* d_input;
    float* d_output;
    int imageSize = img.rows * img.cols * img.channels();
    int outputSize = img.rows * img.cols;

    cudaMalloc( &d_input, imageSize );
    cudaMalloc( &d_output, outputSize * sizeof( float ));

    cudaMemcpy( d_input, img.data, imageSize, cudaMemcpyHostToDevice );

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((img.cols + threadsPerBlock.x - 1) / threadsPerBlock.x,
               (img.rows + threadsPerBlock.y - 1) / threadsPerBlock.y);


    computeLuminance<<<numBlocks, threadsPerBlock>>>(d_input, d_output, img.cols, img.rows);


    float* luminance = new float[ outputSize ];
    cudaMemcpy( luminance, d_output, outputSize * sizeof( float ), cudaMemcpyDeviceToHost );

    // You can now use the 'luminance' array. For instance, print some values:
    for ( int i = 0; i < 10; i ++ ) {
        printf( "%f\n", luminance[i] );
    }

    delete[] luminance;
    cudaFree( d_input );
    cudaFree( d_output );

    return 0;

}

Overwriting luminance.cu


In [9]:
!nvcc -o luminance luminance.cu -I/usr/include/opencv4 -lopencv_core -lopencv_imgcodecs -lopencv_imgproc

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::MultiBandBlender"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially ov

In [10]:
!./luminance

0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000
0.000000


In [11]:
!ls

 drive	     luminance.cu       sample_data
 luminance   luminance.curdir  'Screenshot 2023-08-01 at 5.56.57 PM.png'


In [12]:
!ls luminance.cu

luminance.curdir
